In [ ]:
import os
import math
import logging

import numpy as np
import pandas as pd
import scipy.stats as ss

import matplotlib as mp
%matplotlib inline
import matplotlib.pyplot as plt

from statsmodels.tsa.api import VAR
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, seasonal_plot
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

from IPython.display import display
from tqdm.notebook import tqdm

import utils as ut

In [ ]:
logger = logging.getLogger(__name__)

fmt = '%(asctime)s : %(levelname)s : %(message)s'
logging.basicConfig(format=fmt, level=logging.INFO)

logging.getLogger("gensim").setLevel(logging.WARNING)

In [ ]:
os.chdir(os.path.expanduser('~/github/masthesis/'))

# Load data

In [ ]:
events = pd.read_csv('data/paper-round-3/metadata/event-terms.csv', parse_dates=['timestamp'])
events['date'] = events['timestamp'].dt.date

event_dates = events.groupby('event')['date'].max()

radio_ticks = pd.read_csv('data/paper-round-3/event-annotated/ticks-radio.csv')
radio_ticks['timestamp'] = pd.to_datetime(radio_ticks['timestamp'])

elite_ticks = pd.read_csv('data/paper-round-3/event-annotated/ticks-elite.csv')
elite_ticks['timestamp'] = pd.to_datetime(elite_ticks['timestamp'])

radio_ticks_overall = radio_ticks.loc[
    radio_ticks['is_public'].isna() &
    radio_ticks['station_census_region'].isna() &
    radio_ticks['am_band'].isna() &
    radio_ticks['syndicated'].isna(),
:] \
    .drop(['is_public', 'station_census_region', 'am_band', 'syndicated'], axis=1) \

elite_ticks_overall = elite_ticks.loc[
    elite_ticks['is_retweet'].isna() &
    elite_ticks['conservative'].isna(),
:] \
    .drop(['is_retweet', 'conservative'], axis=1) \

assert radio_ticks_overall.isna().sum().sum() == 0
assert elite_ticks_overall.isna().sum().sum() == 0

In [ ]:
event_cols = list(
    set(c for c in list(radio_ticks) if c.startswith('event_')) &
    set(c for c in list(elite_ticks) if c.startswith('event_'))
)

In [ ]:
for c in event_cols:
    focal_dt = events.loc[events['event'] == c.replace('event_', ''), 'timestamp'].item()
    start_dt = focal_dt - pd.Timedelta(hours=6)
    end_dt = focal_dt + pd.Timedelta(days=4)
    
    radio_ticks_overall['in_window_' + c] = \
        (radio_ticks_overall['timestamp'] >= start_dt) & \
        (radio_ticks_overall['timestamp'] <= end_dt)

    elite_ticks_overall['in_window_' + c] = \
        (elite_ticks_overall['timestamp'] >= start_dt) & \
        (elite_ticks_overall['timestamp'] <= end_dt)

    radio_ticks_overall['in_window_' + c] = radio_ticks_overall['in_window_' + c].astype(int)
    elite_ticks_overall['in_window_' + c] = elite_ticks_overall['in_window_' + c].astype(int)

In [ ]:
radio_ticks_overall = radio_ticks_overall.set_index(['freq', 'timestamp'])
elite_ticks_overall = elite_ticks_overall.set_index(['freq', 'timestamp'])

# Test some econometric assumptions

## Stationarity

The null hypothesis in the augmented Dickey-Fuller test is that there *is* a unit root; if p-value low enough we can reject the existence of a unit root.

In [ ]:
def adf_test(s):
    res = adfuller(s)
    
    stat = {
        'adf_stat': res[0],
        'pval': res[1],
    }
    
    for k, v in res[4].items():
        stat['crit_val_' + str(k)] = v
    
    return stat

In [ ]:
stats = []
for v in event_cols:
    stats += [dict(
        mode='radio',
        event=v,
        **adf_test(radio_ticks_overall.loc[radio_ticks_overall['in_window_' + v] == 1, v]))
    ]
    stats += [dict(
        mode='elite',
        event=v,
        **adf_test(elite_ticks_overall.loc[elite_ticks_overall['in_window_' + v] == 1, v]))
    ]
stats = pd.DataFrame(stats)

stats

## Autocorrelation

### Elite

In [ ]:
nx = 3
ny = int(math.ceil(len(event_cols) / nx))

subplot_size = 5
figsize = (subplot_size * nx, subplot_size * ny)

fig, axes = plt.subplots(ny, nx, figsize=figsize)
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i >= len(event_cols):
        fig.delaxes(ax)
        continue

for dv, ax in zip(event_cols, axes):
    plot_acf(elite_ticks[dv], lags=96, ax=ax)

    ax.set_title(dv)

fig.suptitle('Autocorrelation of mention counts')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

### Radio

In [ ]:
nx = 3
ny = int(math.ceil(len(event_cols) / nx))

subplot_size = 5
figsize = (subplot_size * nx, subplot_size * ny)

fig, axes = plt.subplots(ny, nx, figsize=figsize)
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i >= len(event_cols):
        fig.delaxes(ax)
        continue

for dv, ax in zip(event_cols, axes):
    plot_acf(radio_ticks[dv], lags=96, ax=ax)

    ax.set_title(dv)

fig.suptitle('Autocorrelation of mention counts')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

## Partial autocorrelation

### Elite

In [ ]:
nx = 3
ny = int(math.ceil(len(event_cols) / nx))

subplot_size = 5
figsize = (subplot_size * nx, subplot_size * ny)

fig, axes = plt.subplots(ny, nx, figsize=figsize)
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i >= len(event_cols):
        fig.delaxes(ax)
        continue

for dv, ax in zip(event_cols, axes):
    plot_pacf(elite_ticks[dv], method='ywm', lags=96, ax=ax)

    ax.set_title(dv)

fig.suptitle('Autocorrelation of mention counts')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

### Radio

In [ ]:
nx = 3
ny = int(math.ceil(len(event_cols) / nx))

subplot_size = 5
figsize = (subplot_size * nx, subplot_size * ny)

fig, axes = plt.subplots(ny, nx, figsize=figsize)
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i >= len(event_cols):
        fig.delaxes(ax)
        continue

for dv, ax in zip(event_cols, axes):
    plot_pacf(radio_ticks[dv], method='ywm', lags=96, ax=ax)

    ax.set_title(dv)

fig.suptitle('Autocorrelation of mention counts')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

# Compute VAR models of joint variation

In [ ]:
ic = 'aic'
maxlags = 96 # 15 mins / period * 96 periods = 24 hours

In [ ]:
models, results, irfs = [], [], []
for v in event_cols:
    date = event_dates.to_dict()[v.replace('event_', '')]
    period = '15min'
    
    tmp = pd.concat([
        radio_ticks_overall[v].rename('radio'),
        elite_ticks_overall[v].rename('elite'),
    ], axis=1)
  
    tmp = tmp.loc[pd.IndexSlice[period, :], :] \
        .reset_index() \
        .drop('freq', axis=1)

    tmp = tmp.loc[
        (tmp['timestamp'] >= pd.Timestamp(date, tz='utc') - pd.Timedelta('6h')) &
        (tmp['timestamp'] <= pd.Timestamp(date, tz='utc') + pd.Timedelta('4d')),
    :]
    
    tmp['timestamp'] = tmp['timestamp'].dt.tz_localize(None)
    tmp = tmp.set_index('timestamp').to_period(period)
    
    mod = VAR(tmp)
    res = mod.fit(maxlags=maxlags, ic=ic)
    irf = res.irf(periods=96)
    
    models += [mod]
    results += [res]
    irfs += [irf]

# Test assumptions more

## Selected orders

In [ ]:
for v, mod, res, irf in zip(event_cols, models, results, irfs):
    orders = mod.select_order().selected_orders
    
    print(v)
    print('Selected orders: ', orders)
    print()

## Residual correlations

In [ ]:
for v, mod, res, irf in zip(event_cols, models, results, irfs):
    print(v)
    print(res.resid.corr())
    print()

## Residual autocorrelation: tests

In [ ]:
for v, mod, res, irf in zip(event_cols, models, results, irfs):
    print(v)
    print(res.test_whiteness(nlags=192).summary())
    print()

## Residual autocorrelation: plots

In [ ]:
for v, mod, res, irf in zip(event_cols, models, results, irfs):
    fig, axes = plt.subplots(1, 2, figsize=(6, 3), sharex=True)
    axes = axes.flatten()

    for mode, ax in zip(list(res.resid), axes):
        plot_acf(res.resid[mode], ax=ax, lags=np.arange(1, max(5, orders[ic])))

        ax.set_title(mode)

        ax.set_ylim(-0.05, 0.05)

    fig.suptitle('Residual autocorrelation: ' + v)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

## Residual normality: tests

In [ ]:
for v, mod, res, irf in zip(event_cols, models, results, irfs):
    print(v)
    print(res.test_normality().summary())
    print()

## Residual normality: plots

In [ ]:
for v, mod, res, irf in zip(event_cols, models, results, irfs):
    fig, axes = plt.subplots(1, 2, figsize=(6, 3), sharex=True)
    axes = axes.flatten()

    for mode, ax in zip(list(res.resid), axes):
        res.resid[mode].hist(bins=50, ax=ax, log=True)

        ax.set_title(mode)

    fig.suptitle('Residual autocorrelation: ' + v)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

# Granger causality tests

In [ ]:
dat = []
for v, mod, res, irf in zip(event_cols, models, results, irfs):    
    pvals = pd.DataFrame([
        [v, causing, caused, res.test_causality(caused, causing, kind='wald').pvalue]
        for caused in list(res.resid)
        for causing in list(res.resid)
    ], columns=['event', 'causing', 'caused', 'pval'])
    
    irf_means = pd.DataFrame(irf.irfs.mean(axis=0).T, columns=irf.model.names, index=irf.model.names)
    irf_means = pd.melt(irf_means.reset_index().rename({'index': 'causing'}, axis=1),
                        id_vars='causing', var_name='caused').rename({'value': 'mean'}, axis=1)

    irf_sds = pd.DataFrame(irf.irfs.std(axis=0).T, columns=irf.model.names, index=irf.model.names)
    irf_sds = pd.melt(irf_sds.reset_index().rename({'index': 'causing'}, axis=1),
                      id_vars='causing', var_name='caused').rename({'value': 'std'}, axis=1)

    irf_sems = pd.DataFrame(ss.sem(irf.irfs, axis=0).T, columns=irf.model.names, index=irf.model.names)
    irf_sems = pd.melt(irf_sems.reset_index().rename({'index': 'causing'}, axis=1),
                       id_vars='causing', var_name='caused').rename({'value': 'sem'}, axis=1)

    irf_dat = pvals.merge(irf_means, how='left', on=['causing', 'caused'])
    irf_dat = irf_dat.merge(irf_sds, how='left', on=['causing', 'caused'])
    irf_dat = irf_dat.merge(irf_sems, how='left', on=['causing', 'caused'])

    dat += [irf_dat]
    
dat = pd.concat(dat, axis=0)
dat = dat.loc[dat['causing'] != dat['caused'], :]
dat = dat.sort_values(['event', 'causing', 'caused'])

In [ ]:
with pd.option_context('display.float_format', lambda x: '%.3f' % x, 'display.max_rows', None):
    display(dat)

In [ ]:
dat.groupby(['event', 'causing']).apply(lambda s: s['pval'] < 0.05).reset_index().drop('level_2', axis=1)

In [ ]:
def effect_type(s, alpha=0.05):
    if s['elite'] >= alpha and s['radio'] >= alpha:
        return 'Neither'
    elif s['elite'] < alpha and s['radio'] < alpha:
        return 'T <-> R'
    elif s['elite'] < alpha:
        return 'T -> R'
    else:  # s['radio'] < alpha:
        return 'R -> T'
    
tmp = dat[['event', 'causing', 'pval']].pivot('event', 'causing', 'pval')
tmp.index = tmp.index.to_series().apply(lambda s: s.replace('event_', '').replace('_', ' ').title().replace('Nba', 'NBA'))
tmp['effect'] = tmp.apply(effect_type, axis=1)

tmp = tmp.sort_values('effect')

tmp.columns.name = ''
tmp.index.name = 'Event'
tmp.columns = [s.title() for s in tmp.columns]

with pd.option_context('display.float_format', lambda x: '%.3f' % x, 'display.max_rows', None):
    display(tmp)

In [ ]:
print(tmp.style \
    .format(precision=3) \
    .to_latex(
        hrules = True,
        column_format = 'l|rrl',
        position = 'ht',
        label = 'tab:granger-causality',
        position_float = 'centering',
        environment = 'table',
        convert_css = True,
    )
)

# Impulse responses

In [ ]:
for v, mod, res, irf in zip(event_cols, models, results, irfs):
    _ = irf.plot(impulse='elite', response='radio', orth=True)

    fig = plt.gcf()
    fig.suptitle('Orthogonalized Impulse Responses')

    fig.set_size_inches(5, 5)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    for ax in fig.axes:
        fig.suptitle(v)
        ax.yaxis.set_major_formatter(mp.ticker.PercentFormatter(xmax=1))
        ax.set_ylabel('Estimated Response')
        ax.set_xlabel('Timestep (15-Min Increments)')